# CH04.4. **Bidirectional Sequence Model**

## 00. **작업 환경 설정**

#### 00.0. **사전 변수 설정**

In [1]:
SEED_NUM = 2025
BATCH_SIZE = 512
EPOCH_NUM = 1000
USE_CHECKPOINT_YN = 'N'
MODEL_PTH = '../../model/cifaBiSeq.pt'

#### 00.1. **라이브러리 호출 및 옵션 설정**

In [2]:
#(1) Import libraries
import os
import random
import tqdm
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import sklearn
import torch
import torchvision
import torchinfo

#(2) Set up options
os.environ['PYTHONHASHSEED'] = str(SEED_NUM)
random.seed(a=SEED_NUM)
np.random.seed(seed=SEED_NUM)
torch.use_deterministic_algorithms(mode=True)
torch.manual_seed(seed=SEED_NUM)
torch.mps.manual_seed(seed=SEED_NUM)

#(3) Set up device
if torch.backends.mps.is_available() :
    device = torch.device(device='mps')
else :
    device = torch.device(device='cpu')
print(f'>> Device : {device}')

>> Device : mps


#### 00.2. **사용자정의함수 정의**

In [3]:
#(1) Define `show_img()` function
def show_img(df:torchvision.datasets, index:int) :
    img = df[index][0]
    target = df[index][1]
    img = (img/2+0.5).numpy()
    channel_cnt = img.shape[0]
    if channel_cnt == 3 :
        img = np.transpose(a=img, axes=(1, 2, 0))
        plt.imshow(X=img) 
    elif channel_cnt == 1 : 
        img = np.squeeze(a=img, axis=0)
        plt.imshow(X=img, cmap='gray')
    else : 
        pass 
    plt.xlabel(xlabel=f'Target : {target}({df.classes[target]})')
    plt.show()

#(2) Define `compute_metrics()` function
def compute_metrics(model:torch.nn.Module, loader:torch.utils.data.DataLoader) -> pd.DataFrame :
    preds = []
    targets = []
    model.eval()
    with torch.no_grad() : 
        for inputs, _ in loader :
            preds.extend(
                torch.argmax(input=model(x=inputs), dim=1).to(device='cpu').numpy()
            )
            targets.extend(
                _.to(device='cpu').numpy()
            )
    model.train()
    accuracy = sklearn.metrics.accuracy_score(y_true=targets, y_pred=preds)
    precision = sklearn.metrics.precision_score(y_true=targets, y_pred=preds, average='weighted')
    recall = sklearn.metrics.recall_score(y_true=targets, y_pred=preds, average='weighted')
    f1 = sklearn.metrics.f1_score(y_true=targets, y_pred=preds, average='weighted')
    output = pd.DataFrame(data={
        'metricName' : ['accuracy', 'precision', 'recall', 'f1'], 
        'value'      : [accuracy, precision, recall, f1] 
    })
    return output

#### 00.3. **클래스 정의**

In [4]:
class MyClassificationModel(torch.nn.Module) :
    def __init__(self, input_shape:list, hidden_size:int, layers_num:int, class_num:int, device:torch.device) :
        '''
            input_shape = [channels, height, width] : image shape
        '''
        super().__init__()
        self.seq_len = input_shape[1]               
        self.input_size = input_shape[0] * input_shape[2] # channels * width(or height)
        self.hidden_size = hidden_size
        self.layers_num = layers_num
        self.device = device
        self.layers = torch.nn.ModuleDict()
        self.bi_lstm = torch.nn.LSTM(
            input_size=self.input_size,
            hidden_size=hidden_size,
            num_layers=layers_num,
            batch_first=True,
            bidirectional=True
        )
        self.fc_block = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_size*self.seq_len*2, out_features=class_num),
            torch.nn.LogSoftmax(dim=1)
        )
        self.to(device=self.device)
    def forward(self, x:torch.Tensor) -> torch.Tensor :
        h_0 = torch.zeros(size=(self.layers_num*2, x.shape[0], self.hidden_size), device=self.device)
        c_0 = torch.zeros(size=(self.layers_num*2, x.shape[0], self.hidden_size), device=self.device)
        x = x.permute(dims=(0, 2, 1, 3)) # 차원 순서 변환 (ㅅㅂ)
        x = x.reshape(shape=(x.shape[0], self.seq_len, -1))
        x = x.to(device=self.device)
        x, (h_n, c_n) = self.bi_lstm(x, (h_0, c_0))
        x = x.reshape(shape=(x.shape[0], self.seq_len*self.hidden_size*2))
        x = self.fc_block(x)
        return x

<b></b>

## 01. **데이터셋 전처리 및 로드**

#### 01.1. **이미지 전처리 파이프라인 정의**

In [5]:
img_tf = torchvision.transforms.Compose(
    transforms=[
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]
)

#### 01.2. **데이터셋 로드 및 전처리 적용**

In [6]:
cifa_train = torchvision.datasets.CIFAR10(root='../../data', train=True, download=True, transform=img_tf)
cifa_test = torchvision.datasets.CIFAR10(root='../../data', train=False, download=True, transform=img_tf)

#### 01.3. **EDA**

In [7]:
#(1) Print sample of train
# len(cifa_train)

In [8]:
#(2) Display image
# show_img(df=cifa_train, index=1)

In [9]:
#(3) Check `input_size`
input_shape = list(cifa_train[0][0].shape)

#(4) Print `input_size`
input_shape

[3, 32, 32]

In [10]:
#(5) Print frequency of target class
# target_freq = collections.Counter()
# for i in range(len(cifa_train)):
#     input, target = cifa_train[i]
#     if isinstance(target, torch.Tensor) :
#         target = target.item()
#     target_freq[target] += 1
# pd.DataFrame(data=list(target_freq.items()), columns=['class', 'count']).sort_values(by='class')

#### 01.4. **데이터로더 변환**

In [11]:
cifa_train_loader = torch.utils.data.DataLoader(dataset=cifa_train, batch_size=BATCH_SIZE, shuffle=True)
cifa_test_loader = torch.utils.data.DataLoader(dataset=cifa_test, batch_size=BATCH_SIZE, shuffle=True)

<b></b>

## 02. **모델 구축 및 학습**

#### 02.1. **하이퍼 파라미터 정의**

In [12]:
hidden_size = 32
layers_num = 2

#### 02.2. **모델 정의**

In [13]:
#(1) Define `model`
model = MyClassificationModel(
    input_shape=input_shape, 
    hidden_size=hidden_size, 
    layers_num=layers_num,
    class_num=10,
    device=device
).to(dtype=torch.float32)

#(2) Display `model`
torchinfo.summary(
    model=model, 
    input_size=[BATCH_SIZE]+input_shape,
    device=device
)

Layer (type:depth-idx)                   Output Shape              Param #
MyClassificationModel                    [512, 10]                 --
├─LSTM: 1-1                              [512, 32, 64]             58,368
├─Sequential: 1-2                        [512, 10]                 --
│    └─Linear: 2-1                       [512, 10]                 20,490
│    └─LogSoftmax: 2-2                   [512, 10]                 --
Total params: 78,858
Trainable params: 78,858
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 966.79
Input size (MB): 6.29
Forward/backward pass size (MB): 8.43
Params size (MB): 0.32
Estimated Total Size (MB): 15.04

In [14]:
#(3) Define loss function
criterion = torch.nn.NLLLoss()

#(4) Define optimizer(optimization method)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3, weight_decay=1e-7)

#(5) Define Scheduler
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=30, gamma=0.1)

#### 02.3. **모델 체크포인트 로드**

In [15]:
epoch = 0 
loss_hist = []
best_loss = float('inf')
if USE_CHECKPOINT_YN == 'Y' :
    try :
        checkpoint = torch.load(f=MODEL_PTH, map_location=device)
        model.load_state_dict(state_dict=checkpoint['model'])
        optimizer.load_state_dict(state_dict=checkpoint['optimizer'])
        epoch = checkpoint['best_epoch']
        loss_hist = checkpoint['loss_hist']
        best_loss = loss_hist[-1]
    except Exception :
        pass
print(f">> Epoch={epoch}, Train Loss={best_loss}")

>> Epoch=0, Train Loss=inf


#### 02.4. **모델 학습**

In [ ]:
batch_len = len(cifa_train_loader)
progress_bar = tqdm.trange(epoch+1, EPOCH_NUM+1)
for epoch in progress_bar : 
    last_loss = 0.0
    model.train()
    for inputs, targets in cifa_train_loader :
        optimizer.zero_grad() 
        preds = model(x=inputs)
        loss = criterion(input=preds, target=targets.to(device=device))
        loss.backward()
        optimizer.step()
        last_loss += loss.item()
    last_loss_avg = last_loss / batch_len
    loss_hist.append(last_loss_avg)
    if last_loss_avg < best_loss :
        best_epoch = epoch
        best_loss = last_loss_avg
        torch.save(
            obj={
                'epoch'     : epoch,
                'loss_hist' : loss_hist,
                'model'     : model.state_dict(),
                'optimizer' : optimizer.state_dict()
            }, 
            f=MODEL_PTH
        )
    # scheduler.step()
    progress_bar.set_postfix(ordered_dict={'last_epoch':epoch, 'last_loss':last_loss_avg, 'best_epoch':best_epoch, 'best_loss':best_loss})

 33%|███▎      | 333/1000 [15:00:02<20:02:51, 108.20s/it, last_epoch=333, last_loss=0.00603, best_epoch=322, best_loss=0.00295] 

<b></b>

## 03. **모델 평가**

#### 03.1. **최적 성능 모델 로드**

In [ ]:
checkpoint = torch.load(f=MODEL_PTH, map_location=device)
model.load_state_dict(state_dict=checkpoint['model'])
print(f'>> Best Epoch : {np.argmin(a=checkpoint["loss_hist"])+1}, Best Loss : {np.min(a=checkpoint["loss_hist"])}')

#### 03.2. **과소 적합 확인**

In [ ]:
#(1) Plot training loss
plt.figure(figsize=(12, 6))
plt.xlabel(xlabel='epoch')
plt.ylabel(ylabel='loss')
plt.plot(loss_hist, label='Training Loss')
plt.axvline(x=np.argmin(a=checkpoint["loss_hist"]), color='grey', linestyle='--', linewidth=0.6, label=f'Best Training Loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
#(2) Check metrics
compute_metrics(model=model, loader=cifa_train_loader)

#### 03.3. **일반화 성능 평가**

In [ ]:
compute_metrics(model=model, loader=cifa_test_loader)